In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ruchi798/covid19-pulmonary-abnormalities

In [ ]:
! mkdir spectrograms

In [ ]:
! unzip covid19-pulmonary-abnormalities.zip -d spectrograms

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

dir_data = {}

for dirname, _, filenames in os.walk('./spectrograms/'):
    
    if dirname not in list(dir_data.keys()):
        dir_data[dirname] = []
    for filename in filenames:
        dir_data[dirname].append(filename)
        #os.path.join(dirname, )

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.preprocessing import image
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import * 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.vis_utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pathlib import Path
from skimage.io import imread

In [ ]:
def slice_images(image, IMG_WIDTH, IMG_HEIGHT, parts):
  sliced_img_array = []
  width = int(IMG_WIDTH/parts)
  y = 0
  for x in range(parts):
    y = x*width
    sliced_img_array.append(image[0:IMG_HEIGHT, y:y+width])
  return sliced_img_array

In [ ]:
def create_dataset(img_folders,IMG_WIDTH,IMG_HEIGHT, label):
    img_data_array=[]
    class_name=[]
    n=0
    for dirname, _, filenames in os.walk(img_folders):
        for filename in filenames:
            img_path= os.path.join(dirname, filename)
            # read the image
            image = cv2.imread(img_path)
            # BGR is converted to RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (IMG_WIDTH,IMG_HEIGHT))
            
            #slice the image into 3 equal parts
            images = slice_images(image, IMG_WIDTH, IMG_HEIGHT, 4)
              
            for image in images:
              # convert image to numpy array
              image = np.array(image)
              image = image.astype('float32')
              # normalization
              image /= 255
              img_data_array.append(image)
              class_name.append(label)
              n+=1
    
    print("n= %d",n)
                
    return img_data_array, class_name, n

In [ ]:
IMG_WIDTH = 800
IMG_HEIGHT = 200

noncovid_dir = './spectrograms/Spectrograms/(Real) Non-Covid/'
covid_dir = './spectrograms/Spectrograms/(Real) Covid/'

noncovid_img, noncovid_target, num_noncovid = create_dataset(noncovid_dir, IMG_WIDTH, IMG_HEIGHT, 0)
covid_img, covid_target, num_covid = create_dataset(covid_dir, IMG_WIDTH, IMG_HEIGHT, 1)

In [ ]:
X = noncovid_img + covid_img
y = noncovid_target + covid_target
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [ ]:
plt.figure(figsize=(40,20))
for i in range(6):
    ax=plt.subplot(6,1,(i+1))
    plt.imshow(X[i])
    ax.title.set_text(y[i])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, stratify=y)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, stratify=Y_train)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(300, 300)),
    tf.keras.layers.Dense(600, activation='sigmoid'),
    tf.keras.layers.Dense(2)
])

In [ ]:
num_classes = 2

model = Sequential([
  InputLayer(input_shape=(300, 300)),
  LSTM(600, activation='relu'),
  Dropout(0.4),
  Dense(300, activation='relu'),
  Dropout(0.2),
  Dense(100,activation='sigmoid'),
  Dense(num_classes)
])

In [ ]:
X_train[0].shape

In [ ]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(200,200,1)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
X_train = tf.stack(X_train)
Y_train = tf.stack(Y_train)

In [ ]:
X_valid = tf.stack(X_valid)
Y_valid = tf.stack(Y_valid)

In [ ]:
X_test = tf.stack(X_test)
Y_test = tf.stack(Y_test)

In [ ]:
X_train = tf.expand_dims(X_train,axis= -1)
X_test = tf.expand_dims(X_test,axis= -1)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
#hist = model.fit(X_train, Y_train,batch_size=10, epochs=1,callbacks=[callback])

In [ ]:
model_layers_img = '/tmp/model_layers_img.png'
tf.keras.utils.plot_model(model, to_file=model_layers_img, show_shapes=True)

In [ ]:
history = model.fit(X_train, Y_train,validation_split = 0.2, epochs=3, batch_size=100)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
